# **Inference Model**

In [1]:
# importing libraries
import os
import glob
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.xception import Xception, preprocess_input  
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
def get_f1_score(y_true, y_pred): 
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon()) 
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val

In [11]:
# method to load the model
def load_model():
  model = keras.models.load_model('Xception.h5', custom_objects={'get_f1_score': get_f1_score})
  return model

In [3]:
# loading the testset
def load_test_set(TEST_DIR):
  test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
  test_loader = test_data_generator.flow_from_directory(TEST_DIR,
                                                  target_size = (128, 128),
                                                  color_mode = "rgb",
                                                  shuffle = False,
                                                  class_mode = 'categorical',
                                                  batch_size = 64)
  activity_list = test_loader.class_indices
  return test_loader, activity_list

In [6]:
def get_activity(val, activities):
    for key, value in activities.items():
        if val == value:
            return key

    return "Invalid"

In [8]:
# method to make prediction from whole test set images
def predict_test(loader, model, activity_list):
  filenames = loader.filenames
  samples = len(filenames)
  predictions = model.predict(loader, verbose = 1)

  y_pred = []
  for val in predictions:
      y_pred.append(get_activity(np.argmax(val), activity_list))

  y_true = []
  for file in filenames:
      y_true.append(file.split("\\")[0].split('/')[0])

  print(classification_report(y_pred, y_true))
  return

In [12]:
# main method
if __name__ == '__main__':
  TEST_DIR = r"E:\M_TECH ASSIGNMENTS\Deep Learning\Course Project\final_deliveries\image_testing_set\content\testing_set"
  model = load_model()
  test_set, activity_list = load_test_set(TEST_DIR)

  print('\n')
  
  # if you want to test whole set
  predict_test(test_set, model, activity_list)

Found 7500 images belonging to 50 classes.


 118/7500 [..............................] - ETA: 17:17

7500/7500 [==============================] - 20s 2ms/step
                    precision    recall  f1-score   support

     BaseballPitch       0.91      0.90      0.90       151
        Basketball       0.83      0.80      0.82       156
        BenchPress       1.00      0.94      0.97       159
            Biking       0.85      0.86      0.86       147
         Billiards       1.00      1.00      1.00       150
      BreastStroke       0.98      0.98      0.98       150
      CleanAndJerk       0.93      0.97      0.95       143
            Diving       0.93      0.95      0.94       148
          Drumming       1.00      0.96      0.98       156
           Fencing       0.94      0.94      0.94       150
         GolfSwing       0.89      0.83      0.86       160
          HighJump       0.72      0.77      0.74       141
         HorseRace       0.96      0.96      0.96       150
       HorseRiding       0.89      0.92      0.91       146
          HulaHoop       0.88      0.99  

In [16]:
model.evaluate(test_set)

118/118 [==============================] - 18s 139ms/step - loss: 0.3239 - accuracy: 0.9967 - precision: 0.9645 - recall: 0.8651 - auc: 0.9978 - get_f1_score: 0.9095


[0.32386496663093567,
 0.9966639876365662,
 0.9644715189933777,
 0.865066647529602,
 0.997793436050415,
 0.9095377922058105]